In [2]:
"""
CMA-ES Parameter Optimization for QG Two-Layer Model
Evolution Strategy - No surrogate model needed!
Very robust, can run in parallel
"""

import numpy as np
import pickle
import os
import cma
import matplotlib.pyplot as plt
from qg_model import QGTwoLayerModel
from scipy.ndimage import uniform_filter

# ============================================================================
# COLOR PRINTING
# ============================================================================

class Colors:
    """ANSI color codes for terminal output"""
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    
    @staticmethod
    def green(text):
        return f"{Colors.OKGREEN}{text}{Colors.ENDC}"
    
    @staticmethod
    def red(text):
        return f"{Colors.FAIL}{text}{Colors.ENDC}"
    
    @staticmethod
    def blue(text):
        return f"{Colors.OKBLUE}{text}{Colors.ENDC}"
    
    @staticmethod
    def cyan(text):
        return f"{Colors.OKCYAN}{text}{Colors.ENDC}"
    
    @staticmethod
    def yellow(text):
        return f"{Colors.WARNING}{text}{Colors.ENDC}"
    
    @staticmethod
    def bold(text):
        return f"{Colors.BOLD}{text}{Colors.ENDC}"
    
    @staticmethod
    def star(text):
        return f"{Colors.WARNING}{Colors.BOLD}★ {text}{Colors.ENDC}"

# ============================================================================
# PARAMETER BOUNDS
# ============================================================================

PARAM_BOUNDS = {
    'viscosity_scale': (0.5, 5.0),
    'drag_scale': (0.5, 3.0),
    'eddy_diffusivity': (1e3, 1e5),
    'smagorinsky_coeff': (0.0, 0.3),
    'energy_correction': (-0.01, 0.01),
    'enstrophy_correction': (0.0, 1e-6),
}

PARAM_NAMES = list(PARAM_BOUNDS.keys())
N_PARAMS = len(PARAM_NAMES)

# Transform bounds for CMA-ES (works in [0, 1] space)
BOUNDS_LOWER = np.array([PARAM_BOUNDS[p][0] for p in PARAM_NAMES])
BOUNDS_UPPER = np.array([PARAM_BOUNDS[p][1] for p in PARAM_NAMES])

# ============================================================================
# PARAMETER TRANSFORMATION
# ============================================================================

def transform_to_unit(params):
    """Transform parameters from original space to [0, 1]"""
    return (params - BOUNDS_LOWER) / (BOUNDS_UPPER - BOUNDS_LOWER)

def transform_from_unit(params_unit):
    """Transform parameters from [0, 1] to original space"""
    params = BOUNDS_LOWER + params_unit * (BOUNDS_UPPER - BOUNDS_LOWER)
    # Clip to bounds for safety
    params = np.clip(params, BOUNDS_LOWER, BOUNDS_UPPER)
    return params

# ============================================================================
# SIMULATION RUNNER
# ============================================================================

def run_lowres_with_params(params_array, config_base, highres_results, sim_days=180):
    """
    Run low-res simulation with given parameters and compute loss
    """
    from main_comparison import run_simulation
    
    config = config_base.copy()
    config['subgrid_params'] = {PARAM_NAMES[i]: float(params_array[i]) for i in range(N_PARAMS)}
    
    print(f"\n{'='*70}")
    print(f"Testing parameters:")
    for param_name, val in config['subgrid_params'].items():
        print(f"  {param_name}: {val:.6e}")
    
    try:
        results = run_simulation(config, sim_days=sim_days, save_interval_hours=12)
        loss, detailed_outputs = compute_loss(results, highres_results, return_fields=True)
        
        if not np.isfinite(loss):
            print(Colors.red(f"  ✗ Loss is not finite: {loss}"))
            return np.nan, None, None
        
        print(f"  Loss: {Colors.green(f'{loss:.6f}')}")
        return loss, results, detailed_outputs
        
    except Exception as e:
        print(Colors.red(f"  ✗ Simulation failed: {e}"))
        import traceback
        traceback.print_exc()
        return np.nan, None, None

# ============================================================================
# LOSS COMPUTATION (SAME AS GP)
# ============================================================================

def compute_loss(lowres_results, highres_results, n_days_avg=30, return_fields=False):
    """Compute loss - EXACT SAME AS ML optimization"""
    
    nx_hr = highres_results['config']['nx']
    ny_hr = highres_results['config']['ny']
    nx_lr = lowres_results['config']['nx']
    ny_lr = lowres_results['config']['ny']
    
    coarsen_factor_x = nx_hr // nx_lr
    coarsen_factor_y = ny_hr // ny_lr
    
    times_hr = highres_results['times']
    times_lr = lowres_results['times']
    
    # Use last 30 days for comparison
    time_threshold_hr = times_hr[-1] - n_days_avg
    time_threshold_lr = times_lr[-1] - n_days_avg
    
    indices_hr = np.where(times_hr >= time_threshold_hr)[0]
    indices_lr = np.where(times_lr >= time_threshold_lr)[0]
    
    q1_hr_avg = np.mean([highres_results['q1_history'][i] for i in indices_hr], axis=0)
    q2_hr_avg = np.mean([highres_results['q2_history'][i] for i in indices_hr], axis=0)
    q1_lr_avg = np.mean([lowres_results['q1_history'][i] for i in indices_lr], axis=0)
    q2_lr_avg = np.mean([lowres_results['q2_history'][i] for i in indices_lr], axis=0)
    
    model_hr = highres_results['model']
    model_lr = lowres_results['model']
    
    psi1_hr_avg, psi2_hr_avg = model_hr.q_to_psi(q1_hr_avg, q2_hr_avg)
    psi1_lr_avg, psi2_lr_avg = model_lr.q_to_psi(q1_lr_avg, q2_lr_avg)
    
    H1, H2 = model_hr.H1, model_hr.H2
    H_total = H1 + H2
    
    q_bt_hr = (H1 * q1_hr_avg + H2 * q2_hr_avg) / H_total
    psi_bt_hr = (H1 * psi1_hr_avg + H2 * psi2_hr_avg) / H_total
    q_bt_lr = (H1 * q1_lr_avg + H2 * q2_lr_avg) / H_total
    psi_bt_lr = (H1 * psi1_lr_avg + H2 * psi2_lr_avg) / H_total
    
    def coarsen(field, factor_x, factor_y):
        filtered = uniform_filter(field, size=(factor_y, factor_x), mode='wrap')
        return filtered[::factor_y, ::factor_x]
    
    q_bt_hr_coarse = coarsen(q_bt_hr, coarsen_factor_x, coarsen_factor_y)
    psi_bt_hr_coarse = coarsen(psi_bt_hr, coarsen_factor_x, coarsen_factor_y)
    
    def nrmse(pred, target):
        mse = np.mean((pred - target)**2)
        std = np.std(target)
        return np.sqrt(mse) / (std + 1e-20)
    
    loss_q_bt = nrmse(q_bt_lr, q_bt_hr_coarse)
    loss_psi_bt = nrmse(psi_bt_lr, psi_bt_hr_coarse)
    
    weight_pv = 0.6
    weight_psi = 0.4
    total_loss = weight_pv * loss_q_bt + weight_psi * loss_psi_bt
    
    if return_fields:
        return total_loss, {
            'q_bt_hr_coarse': q_bt_hr_coarse, 'psi_bt_hr_coarse': psi_bt_hr_coarse,
            'q_bt_lr': q_bt_lr, 'psi_bt_lr': psi_bt_lr,
            'loss_q_bt': loss_q_bt, 'loss_psi_bt': loss_psi_bt,
            'total_loss': total_loss, 'weight_pv': weight_pv, 'weight_psi': weight_psi,
        }
    return total_loss

# ============================================================================
# CMA-ES OPTIMIZER
# ============================================================================

class CMAESOptimizer:
    """
    CMA-ES (Covariance Matrix Adaptation Evolution Strategy)
    No surrogate model needed - direct optimization
    """
    
    def __init__(self, initial_params=None, sigma0=0.3, popsize=8):
        """
        Initialize CMA-ES optimizer
        
        Parameters:
        -----------
        initial_params : np.ndarray or None
            Starting point (in original space). If None, use center of bounds
        sigma0 : float
            Initial step size (in unit space, 0.3 = 30% of parameter range)
        popsize : int
            Population size (number of samples per generation)
        """
        # Starting point in unit space
        if initial_params is None:
            # Start at center of parameter space
            x0_unit = np.ones(N_PARAMS) * 0.5
        else:
            x0_unit = transform_to_unit(initial_params)
        
        # CMA-ES options
        opts = {
            'bounds': [0, 1],  # Work in unit space
            'popsize': popsize,
            'seed': 42,
            'verbose': -1,  # Quiet mode, we'll print our own info
        }
        
        self.es = cma.CMAEvolutionStrategy(x0_unit, sigma0, opts)
        
        # Storage
        self.all_samples = []  # List of dicts: {'params', 'loss', 'detailed'}
        self.generation = 0
        self.best_loss = np.inf
        self.best_params = None
        self.best_generation = -1
    
    def optimize(self, config_base, highres_results, max_generations=50):
        """
        Run CMA-ES optimization
        
        Parameters:
        -----------
        config_base : dict
            Base configuration
        highres_results : dict
            High-res results
        max_generations : int
            Maximum number of generations
        """
        print("\n" + "="*70)
        print("CMA-ES OPTIMIZATION")
        print("="*70)
        print(f"Strategy: Evolution Strategy (no surrogate needed)")
        print(f"  Population size: {self.es.popsize}")
        print(f"  Max generations: {max_generations}")
        print(f"  Total evaluations: ~{max_generations * self.es.popsize}")
        print("="*70)
        
        for gen in range(max_generations):
            self.generation = gen
            
            print(f"\n{'='*70}")
            print(f"GENERATION {gen + 1}/{max_generations}")
            print(f"{'='*70}")
            
            # Ask CMA-ES for new population
            solutions_unit = self.es.ask()
            
            # Transform to original parameter space
            solutions = [transform_from_unit(s) for s in solutions_unit]
            
            print(f"Evaluating {len(solutions)} candidates...")
            
            # Evaluate all solutions
            losses = []
            for i, params in enumerate(solutions):
                print(f"\n--- Candidate {i+1}/{len(solutions)} ---")
                
                loss, results, detailed = run_lowres_with_params(
                    params, config_base, highres_results, sim_days=180
                )
                
                losses.append(loss)
                
                # Store sample
                self.all_samples.append({
                    'params': params,
                    'loss': loss,
                    'generation': gen,
                    'detailed': detailed
                })
                
                # Update best
                if np.isfinite(loss) and loss < self.best_loss:
                    self.best_loss = loss
                    self.best_params = params.copy()
                    self.best_generation = gen
                    loss_str = Colors.green(f'{loss:.6f}')
                    print(Colors.star(f"NEW BEST: {loss_str}"))
            
            # Tell CMA-ES the results
            # Replace NaN/inf with large penalty
            losses_clean = [l if np.isfinite(l) else 10.0 for l in losses]
            self.es.tell(solutions_unit, losses_clean)
            
            # Generation summary
            valid_losses = [l for l in losses if np.isfinite(l)]
            n_valid = len(valid_losses)
            n_failed = len(losses) - n_valid
            
            print(f"\n{Colors.bold('Generation Summary:')}")
            print(f"  Valid: {Colors.cyan(str(n_valid))}/{len(losses)}")
            print(f"  Failed: {Colors.red(str(n_failed))}")
            if n_valid > 0:
                gen_best = Colors.green(f'{min(valid_losses):.6f}')
                gen_worst = f'{max(valid_losses):.6f}'
                gen_mean = f'{np.mean(valid_losses):.6f}'
                print(f"  Best this gen:  {gen_best}")
                print(f"  Mean this gen:  {gen_mean}")
                print(f"  Worst this gen: {gen_worst}")
            
            best_str = Colors.green(f'{self.best_loss:.6f}')
            best_gen = Colors.blue(f'(generation {self.best_generation + 1})')
            print(f"  {Colors.bold('Best overall:')} {best_str} {best_gen}")
            
            # Save progress
            self.save_progress()
            
            # Check stopping criteria
            if self.es.stop():
                print(f"\n{Colors.yellow('CMA-ES convergence criteria met!')}")
                break
        
        return self.get_best_params()
    
    def get_best_params(self):
        """Return best parameters as dictionary"""
        if self.best_params is None:
            raise ValueError("No valid parameters found!")
        return {PARAM_NAMES[i]: float(self.best_params[i]) for i in range(N_PARAMS)}
    
    def save_progress(self, filename='cmaes_progress.pkl'):
        """Save optimization progress"""
        data = {
            'all_samples': self.all_samples,
            'best_loss': self.best_loss,
            'best_params': self.best_params,
            'best_generation': self.best_generation,
            'generation': self.generation,
            'es_state': self.es.pickle_dumps(),  # Save CMA-ES state
        }
        with open(filename, 'wb') as f:
            pickle.dump(data, f)
        print(f"  ✓ Progress saved to {filename}")
    
    @classmethod
    def load_progress(cls, filename='cmaes_progress.pkl'):
        """Load optimization progress"""
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        # Create new optimizer and restore state
        optimizer = cls()
        optimizer.all_samples = data['all_samples']
        optimizer.best_loss = data['best_loss']
        optimizer.best_params = data['best_params']
        optimizer.best_generation = data['best_generation']
        optimizer.generation = data['generation']
        
        # Restore CMA-ES state
        optimizer.es = cma.CMAEvolutionStrategy.pickle_loads(data['es_state'])
        
        print(f"✓ Loaded checkpoint:")
        print(f"  Total evaluations: {len(optimizer.all_samples)}")
        print(f"  Generations completed: {optimizer.generation + 1}")
        best_str = Colors.green(f'{optimizer.best_loss:.6f}')
        print(f"  {Colors.bold('Best loss:')} {best_str}")
        
        return optimizer

# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_progress(optimizer, filename='cmaes_progress.png'):
    """Plot optimization progress"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Extract data
    losses = [s['loss'] for s in optimizer.all_samples]
    generations = [s['generation'] for s in optimizer.all_samples]
    valid_mask = np.isfinite(losses)
    
    losses_valid = np.array(losses)[valid_mask]
    generations_valid = np.array(generations)[valid_mask]
    
    # Plot 1: Loss vs evaluation
    ax = axes[0, 0]
    ax.scatter(range(len(losses)), losses, c='blue', alpha=0.5, s=30)
    if optimizer.best_loss < np.inf:
        ax.axhline(optimizer.best_loss, color='green', linestyle='--', linewidth=2,
                  label=f'Best: {optimizer.best_loss:.4f}')
    ax.set_xlabel('Evaluation')
    ax.set_ylabel('Loss')
    ax.set_title('Loss vs Evaluation')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Best loss per generation
    ax = axes[0, 1]
    gen_best = []
    for g in range(optimizer.generation + 1):
        gen_losses = [s['loss'] for s in optimizer.all_samples if s['generation'] == g and np.isfinite(s['loss'])]
        if gen_losses:
            gen_best.append(min(gen_losses))
        else:
            gen_best.append(np.nan)
    
    valid_gen_best = np.isfinite(gen_best)
    ax.plot(np.arange(len(gen_best))[valid_gen_best], 
            np.array(gen_best)[valid_gen_best], 'go-', linewidth=2, markersize=8)
    ax.set_xlabel('Generation')
    ax.set_ylabel('Best Loss')
    ax.set_title('Best Loss Per Generation')
    ax.grid(True, alpha=0.3)
    
    # Plot 3: Loss distribution
    ax = axes[1, 0]
    if len(losses_valid) > 0:
        ax.hist(losses_valid, bins=20, alpha=0.7, color='steelblue', edgecolor='black')
        ax.axvline(optimizer.best_loss, color='green', linestyle='--', linewidth=2, label='Best')
        ax.set_xlabel('Loss')
        ax.set_ylabel('Count')
        ax.set_title('Loss Distribution')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    # Plot 4: Parameter evolution (first parameter)
    ax = axes[1, 1]
    if len(optimizer.all_samples) > 0:
        params_all = np.array([s['params'] for s in optimizer.all_samples])
        param_idx = 0
        param_name = PARAM_NAMES[param_idx]
        
        ax.scatter(range(len(params_all)), params_all[:, param_idx], 
                  c=losses, cmap='viridis_r', alpha=0.6, s=30)
        if optimizer.best_params is not None:
            ax.axhline(optimizer.best_params[param_idx], color='red', linestyle='--', 
                      linewidth=2, label=f'Best: {optimizer.best_params[param_idx]:.4e}')
        ax.set_xlabel('Evaluation')
        ax.set_ylabel(param_name)
        ax.set_title(f'{param_name} Evolution')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.colorbar(ax.collections[0], ax=ax, label='Loss')
    
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"\n✓ Saved: {filename}")
    plt.close()

# ============================================================================
# MAIN
# ============================================================================

def main(checkpoint_file='cmaes_progress.pkl', max_generations=20, popsize=8):
    """Main optimization routine"""
    
    print("\n" + "="*70)
    print("CMA-ES PARAMETER OPTIMIZATION")
    print("="*70)
    
    # Load high-res
    if not os.path.exists('highres_results.pkl'):
        print("\n✗ Error: highres_results.pkl not found!")
        return
    
    with open('highres_results.pkl', 'rb') as f:
        highres_results = pickle.load(f)
    print(f"\n✓ Loaded high-res: {highres_results['config']['nx']}x{highres_results['config']['ny']}")
    
    # Load base config
    from main_comparison import config_lowres
    config_base = config_lowres.copy()
    
    # Load or create optimizer
    if os.path.exists(checkpoint_file):
        print(f"\n✓ Checkpoint found")
        optimizer = CMAESOptimizer.load_progress(checkpoint_file)
    else:
        print(f"\n✓ Starting new optimization")
        optimizer = CMAESOptimizer(sigma0=0.3, popsize=popsize)
    
    # Run optimization
    best_params = optimizer.optimize(config_base, highres_results, max_generations)
    
    # Results
    print("\n" + "="*70)
    print("OPTIMIZATION COMPLETE")
    print("="*70)
    
    print(f"\nTotal evaluations: {len(optimizer.all_samples)}")
    print(f"Generations: {optimizer.generation + 1}")
    
    n_valid = len([s for s in optimizer.all_samples if np.isfinite(s['loss'])])
    n_failed = len(optimizer.all_samples) - n_valid
    print(f"  Successful: {Colors.cyan(str(n_valid))}")
    print(f"  Failed: {Colors.red(str(n_failed))}")
    
    best_str = Colors.green(f'{optimizer.best_loss:.6f}')
    print(f"\n{Colors.bold('Best loss:')} {best_str}")
    print(f"{Colors.bold('Best parameters:')}")
    for name, val in best_params.items():
        val_str = Colors.cyan(f'{val:.6e}')
        print(f"  {name}: {val_str}")
    
    # Save
    with open('cmaes_optimal_params.pkl', 'wb') as f:
        pickle.dump(best_params, f)
    
    with open('cmaes_optimal_config.txt', 'w') as f:
        f.write("'subgrid_params': {\n")
        for name, val in best_params.items():
            f.write(f"    '{name}': {val:.6e},\n")
        f.write("}\n")
    
    print("\n✓ Saved: cmaes_optimal_params.pkl")
    print("✓ Saved: cmaes_optimal_config.txt")
    
    # Plot
    plot_progress(optimizer)
    
    return optimizer, best_params

if __name__ == "__main__":
    # Population size = 8 (8 simulations per generation)
    # 20 generations = 160 total evaluations
    optimizer, best_params = main(max_generations=20, popsize=8)[\


CMA-ES PARAMETER OPTIMIZATION

✓ Loaded high-res: 512x256

✓ Starting new optimization

CMA-ES OPTIMIZATION
Strategy: Evolution Strategy (no surrogate needed)
  Population size: 8
  Max generations: 20
  Total evaluations: ~160

GENERATION 1/20
Evaluating 8 candidates...

--- Candidate 1/8 ---

Testing parameters:
  viscosity_scale: 3.420564e+00
  drag_scale: 1.646301e+00
  eddy_diffusivity: 6.973667e+04
  smagorinsky_coeff: 2.870043e-01
  energy_correction: -1.404967e-03
  enstrophy_correction: 4.297560e-07

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 3.420564106565164
  drag_scale: 1.6463009099661285
  eddy_diffusivity: 69736.67019008865
  smagorinsky_coeff: 0.28700427645209636
  energy_correction: -0.001404967079795481
  enstrophy_correction: 4.2975598614230996e-07

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


100%|██████████| 8640/8640 [00:24<00:00, 347.81it/s]



LowRes_64x32 Simulation Complete!
  Loss: 1.014639
★ NEW BEST: 1.014639

--- Candidate 2/8 ---

Testing parameters:
  viscosity_scale: 4.869231e+00
  drag_scale: 2.325581e+00
  eddy_diffusivity: 3.655638e+04
  smagorinsky_coeff: 1.988316e-01
  energy_correction: -2.780599e-03
  enstrophy_correction: 3.602753e-07

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 4.869231093900247
  drag_scale: 2.325580843351724
  eddy_diffusivity: 36556.37834597429
  smagorinsky_coeff: 0.1988316246980945
  energy_correction: -0.00278059884195808
  enstrophy_correction: 3.602752521857178e-07

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


100%|██████████| 8640/8640 [00:25<00:00, 340.22it/s]



LowRes_64x32 Simulation Complete!
  Loss: 1.090338

--- Candidate 3/8 ---

Testing parameters:
  viscosity_scale: 3.076649e+00
  drag_scale: 5.071933e-01
  eddy_diffusivity: 1.523360e+03
  smagorinsky_coeff: 9.939286e-02
  energy_correction: -6.077189e-03
  enstrophy_correction: 5.942781e-07

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 3.076649066614146
  drag_scale: 0.5071933155229157
  eddy_diffusivity: 1523.359500324758
  smagorinsky_coeff: 0.0993928572055572
  energy_correction: -0.006077189291606752
  enstrophy_correction: 5.94278127952076e-07

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


...


LowRes_64x32 Simulation Complete!
  Loss: 0.354666

--- Candidate 7/8 ---

Testing parameters:
  viscosity_scale: 2.122119e+00
  drag_scale: 1.267367e+00
  eddy_diffusivity: 3.440328e+04
  smagorinsky_coeff: 2.257149e-01
  energy_correction: -1.437152e-03
  enstrophy_correction: 1.747074e-08

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 2.1221192515331984
  drag_scale: 1.2673665302500754
  eddy_diffusivity: 34403.28020705998
  smagorinsky_coeff: 0.22571492720256853
  energy_correction: -0.0014371516328484518
  enstrophy_correction: 1.747074127610847e-08

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


100%|██████████| 8640/8640 [00:26<00:00, 329.21it/s]



LowRes_64x32 Simulation Complete!
  Loss: 0.311452

--- Candidate 8/8 ---

Testing parameters:
  viscosity_scale: 1.864984e+00
  drag_scale: 1.422349e+00
  eddy_diffusivity: 4.610620e+04
  smagorinsky_coeff: 2.332334e-01
  energy_correction: -1.536185e-03
  enstrophy_correction: 7.971749e-10

Running LowRes_64x32 Simulation
Grid: 64 x 32
Resolution: 31.2 km per grid point

Subgrid Parameters:
  viscosity_scale: 1.8649835625625142
  drag_scale: 1.4223489617923375
  eddy_diffusivity: 46106.19882529942
  smagorinsky_coeff: 0.23323336226622132
  energy_correction: -0.0015361852017412789
  enstrophy_correction: 7.971749061777794e-10

Initial Energy: 5.940e+02
Initial Enstrophy: 8.404e-12

Integrating...


100%|██████████| 8640/8640 [00:26<00:00, 331.41it/s]



LowRes_64x32 Simulation Complete!
  Loss: 1.040151

Generation Summary:
  Valid: 8/8
  Failed: 0
  Best this gen:  0.145783
  Mean this gen:  0.399368
  Worst this gen: 1.040151
  Best overall: 0.145783 (generation 20)
  ✓ Progress saved to cmaes_progress.pkl

OPTIMIZATION COMPLETE

Total evaluations: 160
Generations: 20
  Successful: 160
  Failed: 0

Best loss: 0.145783
Best parameters:
  viscosity_scale: 2.610140e+00
  drag_scale: 1.062777e+00
  eddy_diffusivity: 5.187688e+04
  smagorinsky_coeff: 2.203114e-01
  energy_correction: -3.665182e-04
  enstrophy_correction: 2.652481e-09

✓ Saved: cmaes_optimal_params.pkl
✓ Saved: cmaes_optimal_config.txt

✓ Saved: cmaes_progress.png
